In [2]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM

C:\Users\matan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\matan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [3]:

#need to shuffle to respect iid principal for when we split the data
de = DataExploration()

twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()
encoded_speed_dating_df = de.get_hot_encoded_speed_dating(speed_dating_df)

(22223, 10)
(19276, 10)
(41499, 10)
(8378, 44)


C:\Users\matan\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


removing empty values shape:
(6794, 44)


In [32]:
print(speed_dating_df.groupby('decision').agg('count'))

          gender   age  age_o  race  race_o  importance_same_race  \
decision                                                            
0           3863  3863   3863  3863    3863                  3863   
1           2931  2931   2931  2931    2931                  2931   

          importance_same_religion  field  attractive_important  \
decision                                                          
0                             3863   3863                  3863   
1                             2931   2931                  2931   

          sincere_important  ...  clubbing  reading    tv  theater  movies  \
decision                     ...                                             
0                      3863  ...      3863     3863  3863     3863    3863   
1                      2931  ...      2931     2931  2931     2931    2931   

          concerts  music  shopping  yoga  interests_correlate  
decision                                                        
0          

In [24]:
#print(encoded_speed_dating_df.head())
#print(twitter_df.describe())
print(speed_dating_df.head())
print(speed_dating_df.describe())

   gender age age_o                                     race  \
0  female  21    27  'Asian/Pacific Islander/Asian-American'   
1  female  21    22  'Asian/Pacific Islander/Asian-American'   
2  female  21    22  'Asian/Pacific Islander/Asian-American'   
3  female  21    23  'Asian/Pacific Islander/Asian-American'   
4  female  21    24  'Asian/Pacific Islander/Asian-American'   

                                    race_o importance_same_race  \
0              European/Caucasian-American                    2   
1              European/Caucasian-American                    2   
2  'Asian/Pacific Islander/Asian-American'                    2   
3              European/Caucasian-American                    2   
4               'Latino/Hispanic American'                    2   

  importance_same_religion field attractive_important sincere_important  ...  \
0                        4   Law                   15                20  ...   
1                        4   Law                   1

In [ ]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    encoded_speed_dating_df, 'speed_dating')


In [7]:
print(np.any(np.isnan(X_train)))
print(np.all(np.isfinite(X_train)))
print(X_train.shape)

False
True
(4382, 299)


In [9]:
# # data processing and investigation
# TODO for speed dating
# data_twitter = twitter_df.drop('created_at', axis=1).drop('collected_at', axis=1).drop(
#                 'series_of_num_following', axis=1)
# data_twitter['label'] = np.where(data_twitter['is_spam'] == 1, 'spam', 'not_spam')
# data_twitter = data_twitter.drop('is_spam', axis=1)
# pos_twitter = data_twitter.loc[data_twitter['label'] == 'spam'].drop('label', axis=1)
# neg_twitter = data_twitter.loc[data_twitter['label'] == 'not_spam'].drop('label', axis=1)
# 
# create_scatterplot_matrix(data_twitter, "label", 'twitter_df')
# plot_violin_distributions(data_twitter, title='Violin Plots of Twitter Features', dataset_name='twitter_df', label_column='label')
# compare_counts_boxplots(positive_pd=pos_twitter, negative_pd=neg_twitter, title='Boxplots of Twitter Features', positive_label='spam', dataset_name='twitter_df')


In [27]:
def plot_distributions(pd_df, title, dataset_name, cols_to_get_distr=None):
    if cols_to_get_distr == None:
        cols_to_get_distr = pd_df.columns.values
    with plt.style.context('seaborn'):
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 22))
        axes = axes.flatten()

        for col, ax in zip(cols_to_get_distr, axes):
            n = ax.hist(pd_df[col], histtype='bar')
            
            #print(pd_df.groupby(col).agg('count'))
            #print(np.unique(pd_df[col]))
            ax.set_title(col + ' Histogram')
            ax.set(xlabel=str(col + ' Distribution'), ylabel='count of {0}'.format(col))

        plt.suptitle(title + ' Distributions', fontsize=16, verticalalignment='baseline')
        
        # plt.subplots_adjust(left=0.5)
        plt.savefig(f"./figs/data/{dataset_name}_histogram_matrix.png")
        plt.show()


In [28]:
# # data processing and investigation
from utils import create_scatterplot_matrix
print(speed_dating_df.head(3))
cols_of_interest = ['gender', 'age', 'age_o', 'race', 'race_o', 'importance_same_race',
                    'importance_same_religion', 'field', 'attractive_important', 'sincere_important',
                    'intellicence_important', 'funny_important', 'ambtition_important',
                    'shared_interests_important', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition',
                    'attractive_partner', 'sincere_partner', 'intelligence_partner', 'funny_partner',
                    'ambition_partner', 'shared_interests_partner', 'sports', 'tvsports', 'exercise', 'dining',
                    'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies',
                    'concerts', 'music', 'shopping', 'yoga', 'interests_correlate', 'decision']

cols_embedded = ['gender', 'race', 'race_o', 'field']
cols_ints = ['importance_same_race', 'importance_same_religion', 'field', 'attractive_important', 'sincere_important',
             'intellicence_important', 'funny_important', 'ambtition_important',
             'shared_interests_important', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition',
             'attractive_partner', 'sincere_partner', 'intelligence_partner', 'funny_partner',
             'ambition_partner', 'shared_interests_partner', 'sports', 'tvsports', 'exercise', 'dining',
             'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies',
             'concerts', 'music', 'shopping', 'yoga', 'interests_correlate', 'decision']

print(speed_dating_df.shape)
data_speed_dating = speed_dating_df[cols_embedded]
# todo add xlabels
plot_distributions(data_speed_dating, 'Distributions of Speed Dating Categorical Features', 'speed_dating')
data_speed_dating = speed_dating_df[cols_ints]

data_speed_dating['label'] = np.where(data_speed_dating['decision'] == 1, 'positive_date', 'negative_date')
data_speed_dating = data_speed_dating.drop('decision', axis=1)
# pos_twitter = data_twitter.loc[data_twitter['label'] == 'positive_date'].drop('label', axis=1)
# neg_twitter = data_twitter.loc[data_twitter['label'] == 'negative_date'].drop('label', axis=1)
# 
#create_scatterplot_matrix(data_speed_dating, "label", 'speed_dating')
# plot_violin_distributions(data_twitter, title='Violin Plots of Twitter Features', dataset_name='twitter_df', label_column='label')
# compare_counts_boxplots(positive_pd=pos_twitter, negative_pd=neg_twitter, title='Boxplots of Twitter Features', positive_label='spam', dataset_name='twitter_df')


   gender age age_o                                     race  \
0  female  21    27  'Asian/Pacific Islander/Asian-American'   
1  female  21    22  'Asian/Pacific Islander/Asian-American'   
2  female  21    22  'Asian/Pacific Islander/Asian-American'   

                                    race_o importance_same_race  \
0              European/Caucasian-American                    2   
1              European/Caucasian-American                    2   
2  'Asian/Pacific Islander/Asian-American'                    2   

  importance_same_religion field attractive_important sincere_important  ...  \
0                        4   Law                   15                20  ...   
1                        4   Law                   15                20  ...   
2                        4   Law                   15                20  ...   

  reading tv theater movies concerts music shopping yoga interests_correlate  \
0       6  9       1     10       10     9        8    1                0

KeyboardInterrupt: 

In [29]:
#best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7} #twitter
best_params = {'dt__alpha': -0.1, 'dt__max_depth': 9, 'dt__min_samples_leaf': 2} #speed_dating  (CV score=0.724):
dt_clf = DecisionTree(model_name='dt_wip1', dataset_name='speed_dating', X_test=X_test, y_test=y_test, best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth


NameError: name 'X_test' is not defined

In [11]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)
#print(dt_clf.feature_importance_)

{'dt__alpha': -0.1, 'dt__max_depth': 9, 'dt__min_samples_leaf': 2}


In [12]:
# dnn = DNN(n_classes=2, model_name='dnn_wip_3', dataset_name='spam_twitter')
# dnn.train(
#     learning_rate=0.05,
#     steps=1000,
#     batch_size=30,
#     hidden_units=[100, 100],
#     training_examples=X_train_nn,
#     training_targets=y_train_nn,
#     validation_examples=X_validation_nn,
#     validation_targets=y_validation_nn)
# 
# y_pred = dnn.predict(X_test, y_test)

In [12]:
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40} (CV score=0.772):
# todo this is lower than pre pruning, figure out why
boosting_clf = Boosting(model_name='boosting_wip0', dataset_name='speed_dating', X_test=X_test, y_test=y_test)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.772):
{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40}
[[509 125]
 [151 311]]
             precision    recall  f1-score   support

        0.0       0.77      0.80      0.79       634
        1.0       0.71      0.67      0.69       462

avg / total       0.75      0.75      0.75      1096



In [14]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)

{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40}


In [15]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}
knn_clf = KNN(model_name='knn_wip0', dataset_name='speed_dating')
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728):

Best parameter (CV score=0.728):
{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'}
[[527 107]
 [171 291]]
             precision    recall  f1-score   support

        0.0       0.76      0.83      0.79       634
        1.0       0.73      0.63      0.68       462

avg / total       0.74      0.75      0.74      1096



In [17]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)

{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'}


In [ ]:
#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}
#best_params = {'dnn__batch_size': 100, 'dnn__epochs': 50, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
best_params = {'dnn__batch_size': 50, 'dnn__epochs': 3, 'dnn__layer_sizes': [16, 16], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='speed_dating', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)



In [10]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

{'dnn__batch_size': 10, 'dnn__epochs': 1, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'}


In [19]:
# keras_nn_clf = NN_Keras(model_name='keras_nn_optimizer_wip0', dataset_name='spam_twitter')
# keras_nn_clf.train(X_train, y_train, optimizer_tuning=True)
# y_pred = keras_nn_clf.predict(X_test)
# keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [20]:
# from sklearn.svm import SVC
# 
# linear_svm_clf = SVC(kernel='linear') #optimize kernel, regularization, gamma
# linear_svm_clf.fit(X_train, y_train)
# y_pred = linear_svm_clf.predict(X_test)

In [7]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
#{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):

svm_clf_rbf = SVM(model_name='svm_wip_rbf', dataset_name='speed_dating')
svm_clf_rbf.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf.predict(X_test)
svm_clf_rbf.evaluate_model(y_test=y_test, y_pred=y_pred)


Fitting 5 folds for each of 49 candidates, totalling 245 fits
[CV] svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf .......
[CV]  svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf, total=   5.3s
[CV] svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf .......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.0s remaining:    0.0s


[CV]  svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf, total=   5.1s
[CV] svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf .......
[CV]  svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf, total=   5.1s
[CV] svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf .......
[CV]  svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf, total=   5.1s
[CV] svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf .......
[CV]  svm__C=0.00390625, svm__gamma=0.00390625, svm__kernel=rbf, total=   5.2s
[CV] svm__C=0.00390625, svm__gamma=0.015625, svm__kernel=rbf .........
[CV]  svm__C=0.00390625, svm__gamma=0.015625, svm__kernel=rbf, total=   6.4s
[CV] svm__C=0.00390625, svm__gamma=0.015625, svm__kernel=rbf .........
[CV]  svm__C=0.00390625, svm__gamma=0.015625, svm__kernel=rbf, total=   6.5s
[CV] svm__C=0.00390625, svm__gamma=0.015625, svm__kernel=rbf .........
[CV]  svm__C=0.00390625, svm__gamma=0.015625, svm__kernel=rbf, total=   6.1s
[CV] svm__C=0.00390625, svm

[CV] . svm__C=0.015625, svm__gamma=1.0, svm__kernel=rbf, total=   5.9s
[CV] svm__C=0.015625, svm__gamma=1.0, svm__kernel=rbf ................
[CV] . svm__C=0.015625, svm__gamma=1.0, svm__kernel=rbf, total=   6.0s
[CV] svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf ................
[CV] . svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf, total=   6.4s
[CV] svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf ................
[CV] . svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf, total=   6.3s
[CV] svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf ................
[CV] . svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf, total=   6.4s
[CV] svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf ................
[CV] . svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf, total=   6.9s
[CV] svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf ................
[CV] . svm__C=0.015625, svm__gamma=4.0, svm__kernel=rbf, total=   6.2s
[CV] svm__C=0.015625, svm__gamma=16.0, svm__kernel=rbf ...............
[CV]  

[CV] .. svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf, total=   5.7s
[CV] svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf .................
[CV] .. svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf, total=   5.7s
[CV] svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf .................
[CV] .. svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf, total=   5.7s
[CV] svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf .................
[CV] .. svm__C=0.25, svm__gamma=0.0625, svm__kernel=rbf, total=   5.7s
[CV] svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf ...................
[CV] .... svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf, total=   5.8s
[CV] svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf ...................
[CV] .... svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf, total=   5.9s
[CV] svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf ...................
[CV] .... svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf, total=   5.5s
[CV] svm__C=0.25, svm__gamma=0.25, svm__kernel=rbf ...................
[CV] .

[CV] ..... svm__C=1.0, svm__gamma=16.0, svm__kernel=rbf, total=   6.3s
[CV] svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf ..............
[CV]  svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf, total=   3.5s
[CV] svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf ..............
[CV]  svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf, total=   3.5s
[CV] svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf ..............
[CV]  svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf, total=   3.5s
[CV] svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf ..............
[CV]  svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf, total=   3.5s
[CV] svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf ..............
[CV]  svm__C=4.0, svm__gamma=0.00390625, svm__kernel=rbf, total=   3.5s
[CV] svm__C=4.0, svm__gamma=0.015625, svm__kernel=rbf ................
[CV] . svm__C=4.0, svm__gamma=0.015625, svm__kernel=rbf, total=   5.3s
[CV] svm__C=4.0, svm__gamma=0.015625, svm__kernel=rbf ................
[

[CV] ..... svm__C=16.0, svm__gamma=1.0, svm__kernel=rbf, total=   6.4s
[CV] svm__C=16.0, svm__gamma=1.0, svm__kernel=rbf ....................
[CV] ..... svm__C=16.0, svm__gamma=1.0, svm__kernel=rbf, total=   5.9s
[CV] svm__C=16.0, svm__gamma=1.0, svm__kernel=rbf ....................
[CV] ..... svm__C=16.0, svm__gamma=1.0, svm__kernel=rbf, total=   5.8s
[CV] svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf ....................
[CV] ..... svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf, total=   6.2s
[CV] svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf ....................
[CV] ..... svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf, total=   6.2s
[CV] svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf ....................
[CV] ..... svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf, total=   6.2s
[CV] svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf ....................
[CV] ..... svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf, total=   6.2s
[CV] svm__C=16.0, svm__gamma=4.0, svm__kernel=rbf ....................
[CV] .

[Parallel(n_jobs=1)]: Done 245 out of 245 | elapsed: 40.6min finished


Best parameter (CV score=0.781):
{'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}
[[507 130]
 [ 99 360]]
             precision    recall  f1-score   support

        0.0       0.84      0.80      0.82       637
        1.0       0.73      0.78      0.76       459

avg / total       0.79      0.79      0.79      1096



In [9]:
svm_best_params = svm_clf_rbf.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}
# (CV score=0.781):
# {'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}

{'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}


In [ ]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
#{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):

svm_clf_poly = SVM(model_name='svm_wip_poly', dataset_name='speed_dating')
svm_clf_poly.train(X_train, y_train,'poly')
y_pred = svm_clf_poly.predict(X_test)
svm_clf_poly.evaluate_model(y_test=y_test, y_pred=y_pred)


Fitting 5 folds for each of 833 candidates, totalling 4165 fits
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly 
[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly, total=   6.1s
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.5s remaining:    0.0s


[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly, total=   5.9s
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly 
[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly, total=   6.5s
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly 
[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly, total=   8.7s
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly 
[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.00390625, svm__kernel=poly, total=   7.4s
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.015625, svm__kernel=poly 
[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.015625, svm__kernel=poly, total=   6.5s
[CV] svm__C=0.00390625, svm__degree=1, svm__gamma=0.015625, svm__kernel=poly 
[CV]  svm__C=0.00390625, svm__degree=1, svm__gamma=0.015625, svm__kernel=poly, total=   6.2s
[CV] svm__C=0.00390625, svm__degree=1,

In [ ]:
svm_best_params = svm_clf_poly.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
#{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):

svm_clf_linear_sigmoid = SVM(model_name='svm_wip_linear_sigmoid', dataset_name='speed_dating')
svm_clf_linear_sigmoid.train(X_train, y_train,'linear_sigmoid')
y_pred = svm_clf_linear_sigmoid.predict(X_test)
svm_clf_linear_sigmoid.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
svm_best_params = svm_clf_linear_sigmoid.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    twitter_df, 'twitter')


In [ ]:
svm_clf_rbf = SVM(model_name='svm_wip_rbf', dataset_name='twitter')
svm_clf_rbf.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf.predict(X_test)
svm_clf_rbf.evaluate_model(y_test=y_test, y_pred=y_pred)

In [ ]:
svm_best_params = svm_clf_rbf.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}
#Best parameter (CV score=0.893):
#{'svm__C': 16.0, 'svm__gamma': 1.0, 'svm__kernel': 'rbf'}

In [ ]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
#{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):

svm_clf_poly = SVM(model_name='svm_wip_poly', dataset_name='twitter')
svm_clf_poly.train(X_train, y_train,'poly')
y_pred = svm_clf_poly.predict(X_test)
svm_clf_poly.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
svm_best_params = svm_clf_poly.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
#{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):

svm_clf_linear_sigmoid = SVM(model_name='svm_wip_linear_sigmoid', dataset_name='twitter')
svm_clf_linear_sigmoid.train(X_train, y_train,'linear_sigmoid')
y_pred = svm_clf_linear_sigmoid.predict(X_test)
svm_clf_linear_sigmoid.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
svm_best_params = svm_clf_linear_sigmoid.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}
#{'dnn__batch_size': 10, 'dnn__epochs': 1, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='spam_twitter')
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)